In [14]:
import os
import fitz
from PIL import Image
import io
import matplotlib.pyplot as plt

base_dir = os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
pdf_path = "data/NAB/Berichte_NAB 10-025_Kurzarbeitsprogram Geothermiebohrung Schlattingen.PDF"
#pdf_path= "data/test/1062.pdf"
out_dir = os.path.join(base_dir, "data/test2")
pdf_path= "data/NAB/Berichte_NAB 12-040_Untere Rahmengesteine des Opalinustons  Hydrogeologische Einheiten, Gesteinsparam.PDF"

In [22]:
##extract images from document

with fitz.Document(pdf_path) as doc:
    for page_number,page in enumerate(doc):
        image_list = page.get_images()
        drawing_list = page.get_drawings()
        for image_idx, img in enumerate(image_list):
            xref = img[0] 
        # extract image bytes 
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            # get image extension
            image_ext = base_image["ext"]
            
    # Create a PIL Image object from the image bytes
            pil_image = Image.open(io.BytesIO(image_bytes))
            # Save the image to disk
            image_path = os.path.join(base_dir, "data/test2",f"image_{page_number}_{image_idx}.{image_ext}")
            pil_image.save(image_path)




In [23]:
##extract drawings from document

with fitz.open(pdf_path) as doc:
    for page_number,page in enumerate(doc):
        drawings = page.get_drawings()
        if drawings:
            out_doc = fitz.open()
            outpage = out_doc.new_page(width=page.rect.width, height=page.rect.height)
            shape = outpage.new_shape() 
            for draw_idx,drawing in enumerate(drawings):
                for item in drawing["items"]:  # these are the draw commands
                    if item[0] == "l":  # line
                        shape.draw_line(item[1], item[2])
                    elif item[0] == "re":  # rectangle
                        shape.draw_rect(item[1])
                    elif item[0] == "qu":  # quad
                        shape.draw_quad(item[1])
                    elif item[0] == "c":  # curve
                        shape.draw_bezier(item[1], item[2], item[3], item[4])
                    else:
                        raise ValueError("unhandled drawing", item)
                shape.finish(
                    fill=drawing.get("fill", None),  # Fill color
                    color=drawing.get("color", None),  # Line color
                    dashes=drawing.get("dashes", None),  # Line dashing
                )
            shape.commit()
            out_drawing= os.path.join(base_dir, "data/test2", f"drawing_page_{page_number}.pdf")
            out_doc.save(out_drawing)

